In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
sns.set()

from IPython.display import Markdown

In [2]:
TITLE_SIZE = 18

#seeding
rng = np.random.default_rng(42)

In [104]:
# n = number of jobs 
# μ = mean_service_rate
def run_MG1_sim_and_plot(n, small_mean_service_rate, large_mean_service_rate):
    
    #contains all E[T] from the simulation
    sim_mean_response_times = [None] * 9
    
    #contains all lamdas derived from the simulation
    sim_mean_arrival_rates = [None] * 9
    
    #contains all theoretical E[T]
    th_mean_response_times = [None] * 9
    
    #contains all theoretical lamdas
    th_mean_arrival_rates = [None] * 9
    
    #runs the simulation 9 times with utilization equal to 0.1, 0.2, ... , 0.9
    for i in range(1,2):
        lamda = (i/10)/(0.98*(1/small_mean_service_rate) + 0.02*(1/large_mean_service_rate))
        
        #returns a dictionary with: number of jobs, μ, λ, 1/λ, 1/μ
        params = build_MG1_params(n, (0.98*small_mean_service_rate + 0.02*large_mean_service_rate)*(i/10), small_mean_service_rate, large_mean_service_rate)
        #prints the params dictionary using Markdown 
       # display(Markdown(
      #      f"### Simulation: n={n}, " +
      #      f"$\lambda$={format(params['mean_arrival_rate'])}, " +
      #      f"$\mu$={format(params['mean_service_rate'])}, " +
      #      f"$1/\lambda$={format(params['mean_interarrival_time'])}, " +
      #      f"$1/\mu$={format(params['mean_service_time'])}"
      #  ))
        #returns simulation params such as the E[T] and the calculated lamda
        result = run_MG1_sim(params)
        
        sim_mean_response_times[i-1] = result["response_time_mean"]
        sim_mean_arrival_rates[i-1] = result["mean_arrival_rate"]
        
        #E[s]
        th_mean_service_time = 0.98*params["small_mean_service_time"] + 0.02*params["large_mean_service_time"]
        
        #E[s^2]
        th_var_service_time = 0.98*pow(params["small_mean_service_time"], 2) + 0.02*pow(params["large_mean_service_time"],2)
        
        #Ε[Τ] = E[S] + lamda*E[s^2]/(2*(1-ρ))
        th_mean_response_times[i-1] = th_mean_service_time + (lamda*th_var_service_time)/(2*(1-(i/10)))
        th_mean_arrival_rates[i-1] = lamda
        
       # dump_stats(result)
    #plots E[T] as a function of lamda for theoretical and simulated M/M/1's    
    plot_MG1_response_per_lamda(sim_mean_response_times, sim_mean_arrival_rates, th_mean_response_times, th_mean_arrival_rates)

#builds the param dictionary
def build_MG1_params(num_jobs, mean_arrival_rate, small_mean_service_rate, large_mean_service_rate):
    return {
        "n": num_jobs,
        "mean_arrival_rate": mean_arrival_rate,
        "small_mean_service_rate": small_mean_service_rate,
        "large_mean_service_rate": large_mean_service_rate,
        "mean_interarrival_time": 1.0 / mean_arrival_rate,
        "small_mean_service_time": 1.0 / small_mean_service_rate,
        "large_mean_service_time": 1.0 / large_mean_service_rate,
    }

#generates the jobs and their respective arrival, wait, service times using mean_interarrival_time and μ
def run_MG1_sim(params):
    #num of jobs
    n = params["n"]
    
    #service times of jobs
    service_times = np.zeros(n)
    priority = np.zeros(n)
    
    # Parameters
    mean_interarrival_time = params["mean_interarrival_time"]
    small_mean_service_time = params["small_mean_service_time"]
    large_mean_service_time = params["large_mean_service_time"]
    
    # Simulation data and results
    #generating interarrival times using exponential distribution (scale = 1/λ , where λ the rate of the exponential)
    interarrival_times = rng.exponential(scale=mean_interarrival_time, size=n)
    #arrival time of each job is the sum of all the previous interarrival times plus it's own interarrival time
    arrival_times = np.cumsum(interarrival_times)
    
    for i in range(n):
        p = randint(1,100)
        if (p>98):
            #service time for large jobs, μ = 1/201
            service_times[i] = rng.exponential(scale=large_mean_service_time)
            priority[i] = 2 
        else:
            #service time for small jobs μ=1/1
            service_times[i] = rng.exponential(scale=small_mean_service_time)
            priority[i] = 1
            
    #now that we know when every job arrives and how much service time it needs we can find when they exit the queue "start_time"
    #and when they get serviced and exit the system "departure time"
    jobs_df = build_SJF_jobs_df(params, interarrival_times, arrival_times, service_times, priority)
    #total time (secs) needed to service all jobs
    total_width = get_total_width(jobs_df)
    
    sim_mean_interarrival_time = jobs_df["interarrival_time"].mean()
    sim_mean_arrival_rate = 1.0 / sim_mean_interarrival_time
    sim_mean_service_time = jobs_df["service_time"].mean()
    sim_mean_service_rate = 1.0 / sim_mean_service_time
    sim_mean_wait_time = jobs_df["wait_time"].mean()
    sim_response_time_mean = jobs_df["response_time"].mean()
    sim_response_time_var = jobs_df["response_time"].var()
    
    
    
    # utilization
    util = estimate_util(jobs_df)
    
    return {
        "params": params,
        "jobs_df": jobs_df,
        "total_duration": total_width,
        "mean_arrival_rate": sim_mean_arrival_rate,
        "mean_interarrival_time": sim_mean_interarrival_time,
        "mean_service_rate": sim_mean_service_rate,
        "mean_service_time": sim_mean_service_time,
        "mean_wait_time": sim_mean_wait_time,
        "response_time_mean": sim_response_time_mean,
        "utilization": util,
    }


def build_SJF_jobs_df(params, interarrival_times, arrival_times, service_times, priority):    
    n = params["n"]
    
    #jobs dataframe
    jobs_df = pd.DataFrame({
        "interarrival_time": interarrival_times,
        "arrive_time": arrival_times,
        "service_time": service_times,
        "start_time": np.zeros(n),
        "depart_time": np.zeros(n)
    })
    
    #first job gets serviced upon it's arrival time
    jobs_df.loc[0, "start_time"] = jobs_df.loc[0, "arrive_time"]
    #departs after it get's serviced
    jobs_df.loc[0, "depart_time"] = jobs_df.loc[0, "start_time"] + jobs_df.loc[0, "service_time"]
    in_server = 0
    
    
    queue = []
    i = 1
    
    busy = False
    while(0 in jobs_df["depart_time"]):
        print(str(i))
        if (busy == False ):
            
            jobs_df.loc[i, "start_time"] = jobs_df.loc[i, "arrive_time"]
            jobs_df.loc[i, "depart_time"] = jobs_df.loc[i, "start_time"] + jobs_df.loc[i, "service_time"]
            busy = True
        else:
            busy = False
            j = i + 1
            while( jobs_df.loc[j,"arrive_time"] < jobs_df.loc[i, "depart_time"]):
                print("Eomao Vlakas")
                if(jobs_df.loc[j, "depart_time"] == 0):
                    queue.append(j)
                    print("Eimai ligotero vlakas")
                j+=1
            min_service_time = 1000000
            shortest = j
            for k in queue:
                print(str(k))
                if(jobs_df.loc[k, "service_time"] < min_service_time ):
                    min_service_time = jobs_df.loc[k, "service_time"]
                    shortest = k
                
            i = shortest
            to_pop = queue.index(shortest)
            queue.pop(to_pop)
            print(str(len(queue))+" j= "+str(j))
       
        
        
#     queue_start = 0
#     for i in range(1, n):
        
#         if(jobs_df.loc[i, "depart_time"] != 0):
#             continue
#         elif (jobs_df.loc[i, "arrive_time"] >= jobs_df.loc[in_server, "depart_time"] ):
#             print(str(i))
#             jobs_df.loc[i, "start_time"] = jobs_df.loc[i, "arrive_time"]
#             jobs_df.loc[i, "depart_time"] = jobs_df.loc[i, "start_time"] + jobs_df.loc[i, "service_time"]
#             in_server = i
            
#         else :
#             print(str(queue_start))
#             if(queue_start == 0):
#                 queue_start = i
#             for j in range(queue_start,n):
#                 if(jobs_df.loc[j, "arrive_time"] < jobs_df.loc[in_server, "depart_time"]):
#                     if((jobs_df.loc[j, "depart_time"] == 0) and (jobs_df.loc[j, "service_time"] < jobs_df.loc[i, "service_time"]) ):
#                         shortest = j
#                         queue_start = i
                        
#                     elif(jobs_df.loc[i, "depart_time"] == 0):
#                         shortest = i
#                         queue_start = i+1
#                 else:
#                     break
#             jobs_df.loc[shortest, "start_time"] = jobs_df.loc[in_server, "depart_time"]
#             jobs_df.loc[shortest, "depart_time"] = jobs_df.loc[shortest, "start_time"] + jobs_df.loc[shortest, "service_time"]
#             print(str(i)+ " "+str(shortest)+"  "+str(jobs_df.loc[shortest, "depart_time"]))
#             in_server = shortest
#             print(jobs_df["depart_time"])
            
    if (0 in jobs_df["depart_time"] ):
        print("Xasame")
       
    #response time for each job is equal to queueing time plus service time or the time from when it arrives to the system
    #until it gets serviced
    jobs_df["response_time"] = jobs_df["depart_time"] - jobs_df["arrive_time"]
    jobs_df["wait_time"] = jobs_df["start_time"] - jobs_df["arrive_time"]
        
    return jobs_df


#total time system is active is equal to the time that last job finishes minus the arrival time of the 1st job in the system
def get_total_width(jobs_df):
    return jobs_df.iloc[-1]["depart_time"] - jobs_df.iloc[0]["arrive_time"]

#calculates utilization of the system: time the system services jobs / total simulation time
def estimate_util(jobs_df):
    busy = (jobs_df["depart_time"] - jobs_df["start_time"]).sum()
    return busy / get_total_width(jobs_df)

In [4]:
# STATS DUMPS

def format(value):
    return f"{value:,.4f}"

#prints usefull information
def dump_stats(result):
    params = result["params"]
    jobs_df = result["jobs_df"]
    response_time = jobs_df["response_time"]
    arrival_rate_mean = result["mean_arrival_rate"]
    service_rate_mean = result["mean_service_rate"]
    service_time_mean = result["mean_service_time"]
    response_time_mean = result["response_time_mean"]
    util = result["utilization"]
    
    print("Simulation statistics")
    print("---------------------")
    print("total_duration          = " + format(result['total_duration']))
    print("arrival_rate_mean       = " + format(arrival_rate_mean))
    print("interarrival_time_mean  = " + format(result['mean_interarrival_time']))
    # Note: E[response_time] = E[wait_time] + E[service_time]
    print("response_time")
    print("  mean                  = " + format(response_time_mean))
    print("wait_time_mean          = " + format(result['mean_wait_time']))
    print("service_rate_mean       = " + format(service_rate_mean))
    print("service_time_mean       = " + format(service_time_mean))
    print("utilization             = " + format(util))

In [5]:
# PLOTTING FUNCTIONS
#plots E[T] as a function of lamda for Theoretical and Simulation Values
def plot_MG1_response_per_lamda(sim_mean_response_times, sim_mean_arrival_rates, th_mean_response_times, th_mean_arrival_rates):
    
        
    plt.figure(figsize=(14, 2))
    plt.title("E[T] per Lamda", size=TITLE_SIZE)
    plt.xlabel("lamda")
    plt.ylabel("E[T]")
    plt.plot(th_mean_arrival_rates, th_mean_response_times, label="Theoretical", color="red")
    plt.plot(sim_mean_arrival_rates, sim_mean_response_times, label="Simulation", color="blue")
    plt.legend()
    plt.show()

In [105]:
#M/G/1 : 98% service time = 1 sec, 2% service time = 201 secs
run_MG1_sim_and_plot(100, 1, 1/201)

1
1
Eomao Vlakas
Eimai ligotero vlakas
Eomao Vlakas
Eimai ligotero vlakas
2
3
1 j= 4
3
3
2
0 j= 4
2
2


ValueError: 3 is not in list